## Following trips

If it can be assumed that the environmental as well as loading conditions are the same within the small time window between two trips in the same direction with Aurora and Tycho Brahe, the difference in energy consumption between the ships must depend on how the ships are run. We cannot change the weather, but we can change how the ships are run. So if one of the ships is doing it better at a certain trip, it should also be possible to operate the other ship in the same, more optimal way. A comparison between the trips in the same direction that are closest to each other for the two vessels are compared in this section.

In [ ]:
# %load imports.py
# %load ../imports.py
%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as pltb
import matplotlib.pyplot as plt
import seaborn as sns
width=20
height=3
plt.rcParams["figure.figsize"] = (width,height)
sns.set(rc={'figure.figsize':(width,height)})

#import seaborn as sns
import os
from collections import OrderedDict

from IPython.display import display

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)

import sys
import os

from sklearn.metrics import r2_score
import seaborn as sns

import statsmodels.api as sm

from d2e2f.visualization import visualize
import re
from myst_nb import glue

In [ ]:
ship_names = ['tycho','aurora']

df = pd.DataFrame()
for ship_name in ship_names:
    df_ = catalog.load(f'{ship_name}.trip_statistics_joined_thrusters')
    df_['ship'] = ship_name
    df_['start_time'] = pd.to_datetime(df_['start_time'], utc=True)
    df_['end_time'] = pd.to_datetime(df_['end_time'], utc=True)
    
    df = df.append(df_, ignore_index=True)

df.sort_values(by='start_time', inplace=True)

In [ ]:
start = df.iloc[0]['start_time'].date()
end = df.iloc[-1]['start_time'].date()
glue('start',str(start))
glue('end',str(end))

In [ ]:
facegrid = sns.relplot(data=df, x='start_time',y='P', hue='ship', height=3, aspect=3);
glue('fig:tycho-aurora', facegrid.fig, display=False)

{numref}`fig:tycho-aurora` shows the mean power for all trips made by the two ships between {glue:}`start` and {glue:}`end`. The trips are filtered to contain only trips where there exist one trip with Aurora and one trip with Tycho Brahe within the time window of 1 hour as shown in {numref}`fig:tycho-aurora-cut`. It can be seen that there is a period in the begining of May (where Tycho Brahe was taken out of service.) that has been excluded. 

```{glue:figure} fig:tycho-aurora
:figwidth: 800px
:name: "fig:tycho-aurora"

Mean power for all trips and both ships 
```

```{glue:figure} fig:tycho-aurora-cut
:figwidth: 800px
:name: "fig:tycho-aurora-cut"

Mean power for all trips and both ships 
```


In [ ]:
ships = df.groupby(by='ship')
df_tycho=ships.get_group('tycho')
df_aurora=ships.get_group('aurora')
mask = df_aurora['start_time'].apply(lambda x: ((x-df_tycho['start_time']).abs() < "0 days 01:00:00").any())
df_aurora = df_aurora.loc[mask].copy()
df_cut = pd.concat([df_tycho, df_aurora])

In [ ]:
facegrid = sns.relplot(data=df_cut, x='start_time',y='P', hue='ship', height=3, aspect=3);
glue('fig:tycho-aurora-cut', facegrid.fig, display=False)

In [ ]:
sns.displot(df_cut, x='P', hue='ship', kind="kde", bw_adjust=0.5, height=3, aspect=3)

In [ ]:
table_tycho = df_tycho[['sog','P']].describe()
table_tycho.rename(
    columns={'sog':'sog [m/s]',
            'P':'P [kW]',}, inplace=True)

formatter = {
'sog [m/s]' : '{:.2f}',
'P [kW]' : '{:.0f}',
}
glue('tab:tycho',table_tycho.style.format(formatter))

In [ ]:
table_aurora = df_aurora[['sog','P']].describe()
table_aurora.rename(
    columns={'sog':'sog [m/s]',
            'P':'P [kW]',}, inplace=True)

glue('tab:aurora',table_aurora.style.format(formatter))

In [ ]:
P_pct_diff = int(np.round((df_tycho['P'].mean() - df_aurora['P'].mean()) / df_aurora['P'].mean()*100))
glue('P_pct_diff', P_pct_diff)

The statistics in {numref}`tab:tycho` and {numref}`tab:aurora` show that the mean power for all trips is very similar for the two ships. The mean power only differs {glue:}`P_pct_diff`%. The sister ships should therefore have similar possibilites to minimize the energy consumption. Two following trips: one with Aurora and one with Tycho Brahe in the same direction and closest in time can be paired. These pairs can be compared, and since the time window is now very small (less than 1 hour), the difference in energy consumption should come from the operation of the ships. To further contrain the pairs, only pairs where the mean wind speed recorded onboard the ships differs less than {glue:}`w_max_diff` m/s between the ships is included in the anlysis.

```{glue:figure} tab:tycho
:name: "tab:tycho"
Tycho statistics
```

```{glue:figure} tab:aurora
:name: "tab:aurora"
Aurora statistics
```


In [ ]:
df_following = pd.DataFrame()
for trip_direction, df_aurora_ in df_aurora.groupby(by='trip_direction'):
    
    df_tycho_ = df_tycho.groupby(by='trip_direction').get_group(trip_direction)
    
    close_indexes = df_aurora_['start_time'].apply(lambda x: ((x-df_tycho_['start_time']).abs()).idxmin())
    
    df_aurora_.reset_index(inplace=True)
    df_tycho_ = df_tycho_.loc[close_indexes].reset_index()
    
    df = pd.merge(df_aurora_, df_tycho_, how='inner', left_index=True, right_index=True, 
                  suffixes = ('_aurora', '_tycho'))
    df['trip_direction'] = trip_direction    
    df['P_min'] = df[['P_aurora','P_tycho']].min(axis=1)
    df['P_max'] = df[['P_aurora','P_tycho']].max(axis=1)
    df['P_tot'] = df[['P_aurora','P_tycho']].sum(axis=1)
    df['w_diff'] = (df['w_aurora'] - df['w_tycho']).abs()
    df['time_diff'] = (df['start_time_aurora'] - df['start_time_tycho']).abs()
    
    df['P_aurora - P_tycho'] = df['P_aurora'] - df['P_tycho']
    df['sog_aurora - sog_tycho'] = df['sog_aurora'] - df['sog_tycho']
    
    df['energy_saving'] = (df['P_max'] - df['P_min'])
    
    df_following = df_following.append(df)
    

In [ ]:
w_max_diff = 1
glue('w_max_diff',w_max_diff)
mask = ((df_following['w_diff'] < w_max_diff) &
        (df_following['time_diff'] <= '0 days 01:00:00'))



df_following = df_following.loc[mask]
for trip_direction, df in df_following.groupby('trip_direction'):
    facegrid = sns.relplot(data=df, x='P_aurora',y='P_tycho', height=3, aspect=3);
    glue(f'fig:P_following_{trip_direction}', facegrid.fig, display=False)

    facegrid = sns.relplot(data=df, x='sog_aurora - sog_tycho',y='P_aurora - P_tycho', height=3, aspect=3);
    glue(f'fig:sog_following_{trip_direction}', facegrid.fig, display=False)  
    
    facegrid = sns.relplot(data=df, x='w_aurora',y='w_tycho', height=3, aspect=3);
    glue(f'fig:w_following_{trip_direction}', facegrid.fig, display=False)  

The mean power of the following trip paris is shown in {numref}`fig:P_following_Helsingborg-Helsingør` and {numref}`fig:P_following_Helsingør-Helsingborg` for the two directions. The mean power of Aurora (x-axis) is plotted against Tycho Brahe (y-axis). It is very clear that the mean power differs a lot between the ships for trips that are very close in time, where the environment etc. should be very similar. 

In [ ]:
df_energy_saving = df_following.groupby(by='trip_direction')[['energy_saving','P_tot']].sum()
df_energy_saving['energy_saving_pct'] = df_energy_saving['energy_saving']/df_energy_saving['P_tot']*100

df_table = df_energy_saving[['energy_saving_pct']].copy()

df_table.rename(
    columns={'trip_direction':'Direction',
            'energy_saving_pct':'Energy saving [%]',}, inplace=True)

formatter = {
'Energy saving [%]' : '{:.0f}',
}
glue('tab:energy_saving_following',df_table.style.format(formatter))

```{glue:figure} fig:P_following_Helsingborg-Helsingør
:figwidth: 800px
:name: "fig:P_following_Helsingborg-Helsingør"

Comparison of mean power between the two ships for trips (Helsingborg-Helsingør) that are closest in time (1 hour maximum).

```

```{glue:figure} fig:P_following_Helsingør-Helsingborg
:figwidth: 800px
:name: "fig:P_following_Helsingør-Helsingborg"

Comparison of mean power between the two ships for trips (Helsingør-Helsingborg) that are closest in time (1 hour maximum).

```

The energy saving potential is estimated by calculating the amount of energy that could be saved if both vessels are always operated as the better of the two (in the pairs). The result from this calculation is shown in {numref}`tab:energy_saving_following`.

```{glue:figure} tab:energy_saving_following
:name: "tab:energy_saving_following"
Energy saving potential
```

The difference in speed (x-axis) is plotted againast the difference in mean power (y-axis) for the trip pairs in {numref}`fig:sog_following_Helsingborg-Helsingør` and {numref}`fig:sog_following_Helsingør-Helsingborg`. It can be seen from these figures that the mean power difference has a high correlation with the speed difference and therefore seems to be the main explaination to why the energy consumption differs.

```{glue:figure} fig:sog_following_Helsingborg-Helsingør
:figwidth: 800px
:name: "fig:sog_following_Helsingborg-Helsingør"

Comparison of mean power difference and mean speed difference between the two ships for trips (Helsingborg-Helsingør) that are closest in time (1 hour maximum).

```

```{glue:figure} fig:sog_following_Helsingør-Helsingborg
:figwidth: 800px
:name: "fig:sog_following_Helsingør-Helsingborg"

Comparison of mean power difference and mean speed difference between the two ships for trips (Helsingør-Helsingborg) that are closest in time (1 hour maximum).

```


Analysing the difference between pair trips between Aurora and Tycho Brahe estimates that there is an energy saving potential of at least 10%. 